In [ ]:
import os
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0,
    max_output_tokens=2000,
)

class ChatState(TypedDict):
    user_input: str
    intent: str
    answer: str


def classify_intent(state: ChatState) -> Dict:
    user_msg = state["user_input"]
    print(f"\n[classify_intent] User said: {user_msg}")

    prompt = f"""
    You are an intent classifier for a coffee shop chatbot.
    Classify the user's message into one of these categories:
    - price   (asking about cost)
    - product (asking about menu or items sold)
    - other   (anything else)

    Important:
    - Your explanation should NOT be included.
    - Output MUST be only ONE word: price, product, or other.
    - The user will speak Indonesian, but YOU MUST think in English.

    User message: "{user_msg}"
    """

    intent = llm.invoke(prompt).content.strip().lower()
    print("[classify_intent] LLM →", intent)

    return {"intent": intent}


def answer_price(state: ChatState) -> Dict:
    prompt = f"""
    You are a friendly AI barista.

    Your task:
    - Understand the user's Indonesian message: "{state['user_input']}"
    - Answer in NATURAL and FRIENDLY Indonesian.
    - The topic is COFFEE PRICE.
    - Keep the answer concise, accurate, and polite.

    IMPORTANT:
    - Your entire response MUST be in Indonesian.
    """
    answer = llm.invoke(prompt).content.strip()
    return {"answer": answer}


def answer_product(state: ChatState) -> Dict:
    prompt = f"""
    You are a coffee shop assistant.

    Your task:
    - Understand the user's Indonesian message: "{state['user_input']}"
    - Respond in fluent Indonesian.
    - Provide a friendly explanation about menu items (beans, drinks, etc.)
    - Keep the answer short and informative.

    IMPORTANT:
    - Your output MUST be in Indonesian.
    """
    answer = llm.invoke(prompt).content.strip()
    return {"answer": answer}


def answer_other(state: ChatState) -> Dict:
    prompt = f"""
    You are a polite AI barista.

    Your task:
    - Understand the user's Indonesian message: "{state['user_input']}"
    - The question is unclear or outside coffee-related topics.
    - Ask the user to clarify politely.
    - Keep your response SHORT.

    IMPORTANT:
    - Output MUST be in Indonesian.
    """
    answer = llm.invoke(prompt).content.strip()
    return {"answer": answer}


def route_intent(state: ChatState) -> str:
    intent = state["intent"]
    if intent == "price":
        return "price"
    elif intent == "product":
        return "product"
    else:
        return "other"


graph = StateGraph(ChatState)

graph.add_node("classify_intent", classify_intent)
graph.add_node("price", answer_price)
graph.add_node("product", answer_product)
graph.add_node("other", answer_other)

graph.set_entry_point("classify_intent")

graph.add_conditional_edges(
    "classify_intent",
    route_intent,
    {
        "price": "price",
        "product": "product",
        "other": "other",
    }
)

graph.add_edge("price", END)
graph.add_edge("product", END)
graph.add_edge("other", END)

workflow = graph.compile()

def run_bot():
    print("=== CoffeeBot ☕ | Interaction in Indonesian | Prompts in English ===\n")

    while True:
        user_msg = input("Anda: ")

        if user_msg.lower() in ["exit", "quit"]:
            print("CoffeeBot: Sampai jumpa! 👋")
            break

        initial_state = ChatState(
            user_input=user_msg,
            intent="",
           answer=""
        )

        final = workflow.invoke(initial_state)

        print("CoffeeBot:", final["answer"], "\n")

run_bot()


=== CoffeeBot ☕ | Interaction in Indonesian | Prompts in English ===



E0000 00:00:1763214413.396919  112402 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



[classify_intent] User said: halo
[classify_intent] LLM → other
CoffeeBot: Halo! Maaf, saya kurang mengerti. Bisa tolong jelaskan lagi? 


[classify_intent] User said: apa yang bis aanda bantu untuk saya
[classify_intent] LLM → other
CoffeeBot: Mohon maaf, saya kurang mengerti. Ada yang bisa saya bantu terkait kopi? 


[classify_intent] User said: berarti kopi ya?
[classify_intent] LLM → product
CoffeeBot: Tentu saja! "Berarti kopi ya?" itu artinya Anda ingin memesan kopi, kan?

Di sini kami punya banyak pilihan kopi yang enak, mulai dari biji kopi pilihan dari berbagai daerah, sampai berbagai macam racikan minuman kopi yang bisa Anda coba. Ada espresso, latte, cappuccino, manual brew, dan masih banyak lagi.

Ada yang ingin Anda coba atau ada yang mau ditanyakan lebih lanjut? 😊 

CoffeeBot: Sampai jumpa! 👋
